In [2]:
import os
import time
import tqdm
import inceptionv3
import numpy as np
import tensorflow as tf
import defense

/home/stanislas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stanislas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stanislas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stanislas/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

In [4]:
data_path = "./data"
output_path = "./finalresults"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [5]:
cleandata = np.load(os.path.join(data_path, "clean100data.npy"))
cleanlabel = np.load(os.path.join(data_path, "clean100label.npy"))
targets = np.load(os.path.join(data_path, "random_targets.npy"))

In [6]:
xs = tf.placeholder(tf.float32, (299, 299, 3))
l2_x = tf.placeholder(tf.float32, (299, 299, 3))
l2_orig = tf.placeholder(tf.float32, (299, 299, 3))
label = tf.placeholder(tf.int32, ())
one_hot = tf.expand_dims(tf.one_hot(label, 1000), axis=0)

lam = 1e-6
epsilon = 0.05
ENSEMBLE_SIZE = 30
max_steps = 50
LR = 0.1

xs_def = tf.expand_dims(defense.tftensorGD(xs), axis=0)
logits, preds = inceptionv3.model(sess, xs_def)
l2_loss = tf.sqrt(2 * tf.nn.l2_loss(l2_x - l2_orig) / (299 * 299 * 3))

xs_ens = tf.stack([defense.tftensorGD(xs) for _ in range(ENSEMBLE_SIZE)], axis=0)
logits_ens, preds_ens = inceptionv3.model(sess, xs_ens)

labels_ens = tf.tile(one_hot, (logits_ens.shape[0], 1))
xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_ens, labels=labels_ens))
loss_ens = xent + lam * tf.maximum(l2_loss - epsilon, 0)
grad_ens, = tf.gradients(loss_ens, xs)
probs_ens = tf.nn.softmax(logits_ens)


INFO:tensorflow:Scale of 0 disables regularizer.






Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/stanislas/Advanced-Gradient-Obfuscating/data/inception_v3.ckpt
INFO:tensorflow:Scale of 0 disables regularizer.


In [7]:
#adaptive EOT
adv = np.copy(cleandata)
max_steps = 100
acclist = np.zeros((max_steps,))
succrec = np.zeros((max_steps,))
succlist = np.zeros((max_steps,))

start = time.time()
for index in tqdm.tqdm_notebook(range(cleandata.shape[0])):
    
    EOT_adv = np.copy(adv[index])
    print("Generating AE {}...................................".format(index))
    for i in range(max_steps):
        adv_def = defense.defend_FD_sig(defense.defend_FD_sig(EOT_adv))
        #the RDG procedure is inside the session
        p, l2 = sess.run([preds, l2_loss], {xs: adv_def, l2_x: EOT_adv, l2_orig: cleandata[index]})
        acclist[i] += (p == cleanlabel[index]).astype(int)
        succrec[i] += (p == targets[index]).astype(int)
        if p == targets[index] and l2 < epsilon:
            print("Found targeted AE. Iter: {}. L2: {}.".format(i, l2))
            break
        elif l2 > epsilon:
            print("Can't find targeted AE under l2-norm 0.05.")
            break
        
        g_ens, p_ens = sess.run([grad_ens, preds_ens], {xs: EOT_adv, label: targets[index]})
        EOT_adv -= LR * g_ens
        EOT_adv = np.clip(EOT_adv, 0, 1)
    
    if i == max_steps - 1:
        print("Can't find targeted AE under {} iterations for image {}.".format(max_steps, index))
    adv[index] = EOT_adv
for j in range(max_steps):
    succlist[j] = succrec[:j+1].sum()

/home/stanislas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


Generating AE 0...................................
Can't find targeted AE under 100 iterations for image 0.
Generating AE 1...................................
Can't find targeted AE under 100 iterations for image 1.
Generating AE 2...................................
Found targeted AE. Iter: 74. L2: 0.0074470979161560535.
Generating AE 3...................................
Found targeted AE. Iter: 32. L2: 0.008241867646574974.
Generating AE 4...................................
Can't find targeted AE under 100 iterations for image 4.
Generating AE 5...................................
Found targeted AE. Iter: 26. L2: 0.005747338756918907.
Generating AE 6...................................
Can't find targeted AE under 100 iterations for image 6.
Generating AE 7...................................
Found targeted AE. Iter: 10. L2: 0.006491468288004398.
Generating AE 8...................................
Can't find targeted AE under 100 iterations for image 8.
Generating AE 9....................

Can't find targeted AE under 100 iterations for image 75.
Generating AE 76...................................
Can't find targeted AE under 100 iterations for image 76.
Generating AE 77...................................
Can't find targeted AE under 100 iterations for image 77.
Generating AE 78...................................
Can't find targeted AE under 100 iterations for image 78.
Generating AE 79...................................
Can't find targeted AE under 100 iterations for image 79.
Generating AE 80...................................
Found targeted AE. Iter: 57. L2: 0.004996743053197861.
Generating AE 81...................................
Found targeted AE. Iter: 42. L2: 0.006480049807578325.
Generating AE 82...................................
Can't find targeted AE under 100 iterations for image 82.
Generating AE 83...................................
Can't find targeted AE under 100 iterations for image 83.
Generating AE 84...................................
Can't find targe